In [1]:

# coding: utf-8

# In[1]:

import csv

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer , TfidfTransformer
from sklearn.decomposition import NMF
from collections import defaultdict
from bs4 import BeautifulSoup, NavigableString

from lxml.html.soupparser import fromstring
from bs4 import UnicodeDammit

import lxml.etree as ET

import jieba
import jieba.posseg as pseg


jieba.load_userdict("new.dict_all")
# import jieba.analyse
# jieba.analyse.load_stop_words("stop_words_list.txt")


def uri_to_file_name(uri):
    return uri.replace("/", "-")

sessions = {}

xpath_abstract = '''//div[@class='panel-body']/div[1]/text()'''

with open("data/sc.csv", "r") as sessions_file:
    for row in csv.DictReader(sessions_file, ['title', 'link', 'speaker']):  
        session_id = (row['title'])
        filename = "data/sessions/" + uri_to_file_name(row['link']) + '.html'
        page = open(filename).read()
        soup = fromstring(page)
        ab = soup.xpath(xpath_abstract)
        if len(ab) >0 :
            abstract = ab[0].replace('\n', ' ').replace('\r', '').replace('\r\n', '').encode('utf-8','ignore')
            title = row['title']
            sessions[row['link']] = {'title':title , 'abstract':abstract}
           


# In[5]:


        
corpus = []
titles = []
for id, session in sorted(sessions.iteritems(), key=lambda t: t[0]):
    wordlist=pseg.cut(session["abstract"] + session["title"])
    words = ''
    for key in wordlist:  
         words  = words + ' ' + key.word
    corpus.append(words)
    titles.append(session["title"])

n_topics = 10
n_top_words = 10
n_features = 6000

# vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english')
# vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english')


if False :    
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english')
    tfidf =  vectorizer.fit_transform(corpus)
    word = vectorizer.get_feature_names()
    
else :
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english')
    tfidf = vectorizer.fit_transform(corpus)
    word=vectorizer.get_feature_names()



import lda
import numpy as np
# 
vocab = word



if True:
    model = lda.LDA(n_topics=n_topics, n_iter=500, random_state=1)
    model.fit(tfidf)
    topic_word = model.topic_word_
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
        s = ''
        for word in topic_words :
            s = s + ' ' + word.encode('utf-8')
        print('Topic ' + str(i) + ' : ' + s )
else :
    # Fit the NMF model
    print("Fitting the NMF model with n_samples=%d and n_features=%d..."% (tfidf.shape[0], n_features))
    nmf = NMF(n_components=n_topics, random_state=1).fit(tfidf)
    feature_names = vectorizer.get_feature_names()
    
    for topic_idx, topic in enumerate(nmf.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()


# 
# doc_topic = model.doc_topic_
# for i in range(0, len(titles)):
#     print("{} (top topic: {})".format(titles[i], doc_topic[i].argmax()))
#     print(doc_topic[i].argsort()[::-1][:3])






Building prefix dict from /usr/local/lib/python2.7/dist-packages/jieba/dict.txt ...
DEBUG:jieba:Building prefix dict from /usr/local/lib/python2.7/dist-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/jieba/__init__.py", line 103, in initialize
    replace_file(fpath, cache_file)
OSError: [Errno 1] Operation not permitted
ERROR:jieba:Dump cache file failed.
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/jieba/__init__.py", line 103, in initialize
    replace_file(fpath, cache_file)
OSError: [Errno 1] Operation not permitted
Loading model cost 1.20809912682 seconds.
DEBUG:jieba:Loading model cost 1.20809912682 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Pref

Topic 0 :  語音 我們 文字 服務 django 位置 方式 套件 解決
Topic 1 :  python 如何 使用 一個 問題 如果 設計 經驗 eda
Topic 2 :  applications create static map api use time google interface
Topic 3 :  data network information mobile fields analytics protocol practice learn
Topic 4 :  介紹 hdf5 ceph 利用 測試 ipython 存取 notebook pytest
Topic 5 :  python sphinx programming debugger use language explain documentation process
Topic 6 :  swift developers nodes benchmark cloud building service ansible cluster
Topic 7 :  technology lcd program happy designers programmers order provides tasks
Topic 8 :  python talk like analysis used learning use tools ll
Topic 9 :  using make gandi manage development pycon services physics session
